In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
dbutils.fs.ls("abfss://landing@salesmarketingstorage1.dfs.core.windows.net")

In [0]:
df = spark.read \
.options(header=True) \
.csv("abfss://landing@salesmarketingstorage1.dfs.core.windows.net/orders")

df.display(10)

In [0]:
df_orders = (
    spark.readStream 
    .format("cloudFiles").option("cloudFiles.format", "csv")
    .option("header", "true")
    .option("quote", "\"")
    .option("mode", "PERMISSIVE")
    .option("cloudFiles.schemaHints", "order_purchase_timestamp TIMESTAMP, order_approved_at TIMESTAMP, order_delivered_carrier_date TIMESTAMP, order_delivered_customer_date TIMESTAMP, order_estimated_delivery_date TIMESTAMP")
    .option("cloudFiles.schemaLocation", "abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/bronze/orders")
    .load("abfss://landing@salesmarketingstorage1.dfs.core.windows.net/orders/")
    .writeStream
    .format("delta")
    .option("checkpointLocation", "abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/checkpoints/bronze/orders") 
    .table("bronze.orders")
)

In [0]:
%sql
describe sales_marketing_ws.bronze.orders;

In [0]:
%sql
SELECT * FROM cloud_files_state('abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/checkpoints/bronze/orders');

In [0]:
%sql
select * from sales_marketing_ws.bronze.orders limit 10;